Given a sample

In [1]:
sample = {"input":"\n  ['این', 'اولین', 'قرارداد', 'میان', 'شرکت', 'ملی', 'نفت', 'چین', 'با', 'ایران', 'از', 'هنگام', 'پیروزی', 'انقلاب', 'اسلامی', 'است', '.']\n  ","output":"[('این', 'O'), ('اولین', 'O'), ('قرارداد', 'O'), ('میان', 'O'), ('شرکت', 'ORG'), ('ملی', 'ORG'), ('نفت', 'ORG'), ('چین', 'ORG'), ('با', 'O'), ('ایران', 'LOC'), ('از', 'O'), ('هنگام', 'O'), ('پیروزی', 'O'), ('انقلاب', 'O'), ('اسلامی', 'O'), ('است', 'O'), ('.', 'O')]"}

# Prepare input prompt

In [2]:
from prompt import ENGLISH_ONE

In [3]:
sample_prompt = ENGLISH_ONE.format(input=sample['input'])

In [4]:
print(sample_prompt)


  Task Description:
  You need to label a given Persian token list with named entity labels.
  Named Entity Labels:

  PER (person)
  LOC (location)
  ORG (organization)
  Product (Product)
  Event (Event)
  Facility (Facility)

  Here, we provide you examples of the task:
    

  Example 1:
      Input: ['حمید', 'طاهایی', 'افزود', ':', 'برای', 'اجرای', 'این', 'طرحها', '0', 'میلیارد', 'و', '0', 'میلیون', 'ریال', 'اعتبار', 'هزینه', 'شده', 'است', '.']
      Output: [('حمید', 'PER'), ('طاهایی', 'PER'), ('افزود', 'O'), (':', 'O'), ('برای', 'O'), ('اجرای', 'O'), ('این', 'O'), ('طرحها', 'O'), ('0', 'O'), ('میلیارد', 'O'), ('و', 'O'), ('0', 'O'), ('میلیون', 'O'), ('ریال', 'O'), ('اعتبار', 'O'), ('هزینه', 'O'), ('شده', 'O'), ('است', 'O'), ('.', 'O')]

  

  Example 2:
      Input: ['جشنواره', 'فیلم', 'دفاع', 'مقدس', 'از', '0', 'تا', '0', 'مهرماه', 'در', 'چهار', 'سینمای', 'تهران', 'و', 'فرهنگسرای', 'نیاوران', 'و', 'بهمن', 'بر', 'پا', 'می‌شود', '.']
      Output: [('جشنواره', 'Event'), ('فیلم',

# Evaluate model's response

In [5]:
sample_GPT_response = """[('این', 'O'), ('اولین', 'O'), ('قرارداد', 'O'), ('میان', 'O'), ('شرکت', 'O'), ('ملی', 'ORG'), ('نفت', 'ORG'), ('چین', 'LOC'), ('با', 'O'), ('ایران', 'LOC'), ('از', 'O'), ('هنگام', 'O'), ('پیروزی', 'O'), ('انقلاب', 'O'), ('اسلامی', 'Event'), ('است', 'O'), ('.', 'O')]"""

In [6]:
tags = ["O", 'PER', 'person', 'LOC', 'location', 'ORG', 'organization', 'Product', 'Event', 'Facility', "I-event", "I-fac", "I-loc", "I-org", "I-pers", "I-pro", "B-event", "B-fac", "B-loc", "B-org", "B-pers", "B-pro"]
tags_2_i = {v:i for i,v in enumerate(tags)}
i_2_tags = {i:v for i,v in enumerate(tags)}

In [7]:
def convert_2_list(l12):
  l1 = l12[l12.index('['):]
  # print(l1)
  if (l1[0] != '[' or l1[-1] != ']') and len(l1) > 2:
    return [('گفت', 'O')]
  l1 = l1.replace('...',"', ''")
  l1 = l1.replace('\'',"\"")
  # print(l1)
  res = l1.strip('][').split(', ')
  out = []
  for i in range(0,len(res)-1,2):
    a = res[i].replace('\"','')
    a = a.replace('(','')
    a = a.replace(')','')
    b = res[i+1].replace('\"','')
    b = b.replace('(','')
    b = b.replace(')','')
    out.append((a,b))
  return out
convert_2_list("[('یکی', 'O'), ('از', 'O'), ('نمایندگان', 'O'), ('در', 'O'), ('این', 'O'), ('خصوص', 'O'), ('به', 'O'), ('خبرنگار', 'O'), ('ما', 'O'), ('گفت', 'O'), (':', 'O'), ('بیشترین', 'O'), ('نگرانی\u200cها', 'O'), ('مربوط', 'O'), ('به', 'O'), ('بازداشتگاه\u200cها', 'O'), ('است', 'O'), ('.', 'O')]")

[('یکی', 'O'),
 ('از', 'O'),
 ('نمایندگان', 'O'),
 ('در', 'O'),
 ('این', 'O'),
 ('خصوص', 'O'),
 ('به', 'O'),
 ('خبرنگار', 'O'),
 ('ما', 'O'),
 ('گفت', 'O'),
 (':', 'O'),
 ('بیشترین', 'O'),
 ('نگرانی\u200cها', 'O'),
 ('مربوط', 'O'),
 ('به', 'O'),
 ('بازداشتگاه\u200cها', 'O'),
 ('است', 'O'),
 ('.', 'O')]

In [8]:
def findstem(arr):

    # Determine size of the array
    n = len(arr)

    # Take first word from array
    # as reference
    s = arr[0]
    l = len(s)

    res = ""

    for i in range(l):
        for j in range(i + 1, l + 1):

            # generating all possible substrings
            # of our reference string arr[0] i.e s
            stem = s[i:j]
            k = 1
            for k in range(1, n):

                # Check if the generated stem is
                # common to all words
                if stem not in arr[k]:
                    break

            # If current substring is present in
            # all strings and its length is greater
            # than current result
            if (k + 1 == n and len(res) < len(stem)):
                res = stem

    return res


In [9]:
def comparable_gold_pred2(res, ls, ts):
  # print(res)
  # print(ts)
  # print([i_2_tags[i] for i in ls])
  i2l = [i_2_tags[i] for i in ls]
  for i, v in enumerate(ts):
    # print(v)
    # if i < len(res):
      # print(res[i])
    # else:
    #   res.append(("", ''))
    if i >= len(res):
      res.append(("", ''))
      # print(res[i])
    # print(i2l[i])
    # print(res[i][0].replace("\\u200c"," "), ts[i].replace("\u200c"," "))
    # print('dataset', ts[i].replace("\u200c"," ").lower())
    # print('result', res[i][0].replace("\\u200c"," ").lower())
    # print('stem', findstem([res[i][0].replace("\\u200c"," ").lower(), ts[i].replace("\u200c"," ").lower()]))
    if findstem([res[i][0].replace("\\u200c"," ").lower(), ts[i].replace("\u200c"," ").lower()]) == ts[i].replace("\u200c"," ").lower():
      if res[i][1].lower() in i2l[i].lower():
        # print(i, 'match', i2l[i], 'word', ts[i])
        tempi = list(res[i])
        tempi[1] = i2l[i]
        res[i] = tuple(tempi)
    else:
      # print('yyyyyy')
      # print(res, i2l)
      tempi = list(res[i])
      tempi[1] = 'O'
      res[i] = tuple(tempi)
      continue
  # print('res len', len(res), 'inp len', len(ts), 'labels len', len(ls))
  # print([i[1] for i in res])
  # print(i2l)
  for i,v in enumerate(res):
    # print(v, len(i2l), i)
    if i >= len(i2l):
      break
    if v[1] != i2l[i] and not v[1] in tags:
      for k,j in enumerate(tags):
        if v[1].lower() in j.lower():
          temp = list(res[i])
          temp[1] = tags[k]
          res[i] = tuple(temp)
  return [i[1] for i in res], i2l

In [10]:
gls = []
pls = []

ls = ['PER', 'person', 'LOC', 'location', 'ORG', 'organization', 'Product', 'Event', 'Facility']

for i in range(1):

  g = convert_2_list(sample['output'])
  p = convert_2_list(sample_GPT_response)

  pl, gl = comparable_gold_pred2(p, [(tags_2_i[i[1]] if i[1] in ls else tags_2_i['O'])  for i in g], [i[0] for i in g])
  print(pl, gl, [i[0] for i in g])
  gls.extend([(i if i in ls else 'O') for i in gl])
  pls.extend([(i if i in ls else 'O') for i in pl][:len(gl)])
  # break

['O', 'O', 'O', 'O', 'ORG', 'ORG', 'ORG', 'LOC', 'O', 'LOC', 'O', 'O', 'O', 'O', 'Event', 'O', 'O'] ['O', 'O', 'O', 'O', 'ORG', 'ORG', 'ORG', 'ORG', 'O', 'LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O'] ['این', 'اولین', 'قرارداد', 'میان', 'شرکت', 'ملی', 'نفت', 'چین', 'با', 'ایران', 'از', 'هنگام', 'پیروزی', 'انقلاب', 'اسلامی', 'است', '.']


In [11]:
goi = []
pri = []

for i in gls:
  goi.append(tags_2_i[i])

for i in pls:
  pri.append(tags_2_i[i])

In [12]:
from sklearn.metrics import classification_report

print(classification_report(goi, pri, target_names=[tags[i] for i in set(goi).union(set(pri))], digits=6)) #target_names=[tags[i] for i in [0,1,3,5,7,8,9]],

              precision    recall  f1-score   support

           O   1.000000  0.916667  0.956522        12
         LOC   0.500000  1.000000  0.666667         1
         ORG   1.000000  0.750000  0.857143         4
       Event   0.000000  0.000000  0.000000         0

    accuracy                       0.882353        17
   macro avg   0.625000  0.666667  0.620083        17
weighted avg   0.970588  0.882353  0.916088        17



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
